#Some main steps:

 Download dbNSFP1.3_a 
 
 Use pseq to annotation all positions inside this database
 
 Convert to VCF
 
 Extract variants


Notes:
In dbNSFP, there are positions whose ref and alt are the same, 

$=>$ pseq: we cannot get "snp_fw_seq3" (REF = ALT)

Ex:

chrM:8531	allele_ref	A

chrM:8531	allele_alt	A

chrM:8531	seqdb_ref	A

chrM:8531	func_alt	.

chrM:8531	func	intergenic-region

chrM:8531	transcript	.

chrM:8531	transcript_strand	.


Some small differences between Old and New data sets

result_mutation_gene_damaging.7methods.txt: use old gene names

Result_7methods_allGenes_trinucleotide.txt.join.192_mutationRate_forGenes: new gene names


## Convert to an input file for pseq

zcat db.shortened.dbnsfp31a.gz|awk 'NR>1 {print $1":"$2,$5,$6,$7}' > all_coordinate_hg19_dbnsfp31a.txt &

## Add chromosomes

awk -F":" '{print $1, $0}' all_coordinate_hg19_dbnsfp31a.txt > all_coordinate_hg19_dbnsfp31a.txt_singleChr.txt

## Split into chromosomes

In R

library("data.table")
aa <- fread("all_coordinate_hg19_dbnsfp31a.txt_singleChr.txt", header = FALSE)

setkey(aa, V1)

x2 <- table(x1)
x2 <- names(x2)

for (ii in (1:length(x2))) {
 xTemp <- aa[x2[ii]]
 write.table(xTemp, paste("SingleChr_annotation/All_coordinate_chr", x2[ii], ".txt", sep = ""), col.names=FALSE, row.names=FALSE, quote=FALSE) }

## Extract trinucleotide context (fast)

for file in $(ls Reduced.All_coordinate_chr*annotated|grep -v "chr2\.")

do ./convert2columns.sh $file

done 


## Create vcf files (fast)

################
Use the file 

convert2VCF_and_filter.sh

## Combine all files

outFile=Result_7methods_allGenes_trinucleotide.txt

rm -f ${outFile}

for file in $(ls Extract.damaging.7methods.Reduced.All_coordinate_chr*)
do

grep -v "#" $file|awk -F";" '{print $1}'|awk 'NF>7'|\
    awk '{print $(NF-1),$NF}'|sort|\
    uniq -c|awk '{print $3,$2,$1}' >> ${outFile}
    
done


## Join 192

cat Result_7methods_allGenes_trinucleotide.txt |sort|join -1 1 -2 1 - ~/psychen/resources/genic_mutation_w_annotation/192_trinuc_mutations.sort.txt > Result_7methods_allGenes_trinucleotide.txt.join.192
